In [ ]:
!pip install google-generativeai requests beautifulsoup4 pandas

In [15]:
import google.generativeai as genai
import os
import pandas as pd
import re
import requests

from bs4 import BeautifulSoup
from dotenv import load_dotenv

## 설정 (Configuration)

설정 로딩

In [4]:
load_dotenv()  # .env 파일 로드

GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
if not GEMINI_API_KEY:
    raise ValueError("GEMINI_API_KEY not found in environment variables.")

Gemini 2.5 Flash 모델

In [17]:
genai.configure(api_key=GEMINI_API_KEY)

# 모델 설정 (긴 문서를 처리하기 위해 Gemini 2.5 Flash 또는 Pro 권장)
#model = genai.GenerativeModel('gemini-2.5-flash')
model = genai.GenerativeModel('gemini-2.5-pro')

SEC 요청 헤더 설정

In [6]:
# SEC 요청을 위한 헤더 (필수: 이름과 이메일을 포함해야 함)
HEADERS = {
    "User-Agent": "Sayouzone sjkim@sayouzone.com"
}

SEC EDGAR 데이터 가져오기

- ticker으로 CIK (Central Index Key) 확인 (https://www.sec.gov/files/company_tickers.json)

In [29]:
# ==========================================
# 2. SEC EDGAR 데이터 가져오기 함수
# ==========================================
def get_cik_from_ticker(ticker: str):
    """
    티커(Ticker)를 사용하여 SEC 고유 식별자(CIK)를 찾습니다.
    
    Args:
        ticker (str): 티커(Ticker) 예: NVDA, AAPL, TSLA, GOOG, MSFT, AMZN 등
    
    Returns:
        str: CIK 반환 
        예: AAPL -> 320193 -> 0000320193
        NVDA -> 1045810 -> 0001045810
    """
    
    url = "https://www.sec.gov/files/company_tickers.json"
    response = requests.get(url, headers=HEADERS)
    data = response.json()
    
    # 데이터 프레임으로 변환하여 검색
    df = pd.DataFrame.from_dict(data, orient='index')
    target = df[df['ticker'] == ticker.upper()]
    
    if target.empty:
        raise ValueError(f"Ticker '{ticker}' not found.")
    
    # CIK는 10자리 숫자로 패딩해야 함 (예: 320193 -> 0000320193)
    print(f"CIK : {target.iloc[0]['cik_str']} {str(target.iloc[0]['cik_str']).zfill(10)}")
    return str(target.iloc[0]['cik_str']).zfill(10)        

def get_latest_10k_text(ticker: str):
    """
    특정 기업의 최신 10-K 보고서 텍스트를 가져옵니다.

    기업의 제출 내역(Submissions) JSON 가져오기
    예: https://data.sec.gov/submissions/CIK0000320193.json
    https://data.sec.gov/submissions/CIK0001045810.json

    최신 10-K URL
    예: https://www.sec.gov/Archives/edgar/data/0000320193/000032019325000079/aapl-20250927.htm
    https://www.sec.gov/Archives/edgar/data/0001045810/000104581025000023/nvda-20250126.htm
    
    Args:
        ticker (str): 티커(Ticker) 예: NVDA, AAPL, TSLA, GOOG, MSFT, AMZN 등

    Returns:
        str: 10-K 문서에서 텍스트 추출 및 불팰요한 공백 제거
    """
    
    try:
        cik = get_cik_from_ticker(ticker)
        print(f"[*] '{ticker}'의 CIK: {cik}")
        
        # 기업의 제출 내역(Submissions) JSON 가져오기
        submissions_url = f"https://data.sec.gov/submissions/CIK{cik}.json"
        response = requests.get(submissions_url, headers=HEADERS)
        data = response.json()
        
        # 최근 제출 목록에서 10-K 찾기
        filings = data['filings']['recent']
        df_filings = pd.DataFrame(filings)
        print(df_filings)
        
        # 10-K만 필터링하고 가장 최신 것 선택
        latest_10k = df_filings[df_filings['form'] == '10-K'].iloc[0]
        latest_10q = df_filings[df_filings['form'] == '10-Q'].iloc[0]
        
        accession_number = latest_10k['accessionNumber'].replace('-', '')
        primary_document = latest_10k['primaryDocument']

        # 실제 문서 URL 생성
        doc_url = f"https://www.sec.gov/Archives/edgar/data/{cik}/{accession_number}/{primary_document}"
        print(f"[*] 최신 10-K URL: {doc_url}")

        accession_number = latest_10q['accessionNumber'].replace('-', '')
        primary_document = latest_10q['primaryDocument']
        
        # 실제 문서 URL 생성
        doc_url = f"https://www.sec.gov/Archives/edgar/data/{cik}/{accession_number}/{primary_document}"
        print(f"[*] 최신 10-Q URL: {doc_url}")
        
        # 문서 내용 가져오기
        doc_response = requests.get(doc_url, headers=HEADERS)
        
        # HTML 파싱 및 텍스트 추출
        soup = BeautifulSoup(doc_response.content, 'html.parser')
        text = soup.get_text(separator='\n')
        
        # 불필요한 공백 제거
        clean_text = re.sub(r'\n+', '\n', text).strip()
        return clean_text
        
    except Exception as e:
        print(f"[!] Error fetching data: {e}")
        return None

In [18]:
# ==========================================
# 3. Gemini 분석 함수
# ==========================================
def analyze_sec_filing(ticker):
    print(f"--- {ticker} 10-K 데이터 수집 중... ---")
    doc_text = get_latest_10k_text(ticker)
    
    if not doc_text:
        return

    print(f"--- Gemini 분석 시작 (텍스트 길이: {len(doc_text)} 자) ---")
    
    # 프롬프트 작성 (원하는 분석 관점에 따라 수정 가능)
    prompt = f"""
    당신은 전문 금융 애널리스트입니다. 아래 제공된 {ticker}의 SEC 10-K 보고서 텍스트를 분석하여 다음 내용을 한국어로 요약해 주세요.
    
    1. **재무 하이라이트**: 매출, 순이익 등 주요 재무 성과의 추세.
    2. **주요 리스크 요인(Risk Factors)**: 경영진이 언급한 가장 치명적인 리스크 3가지.
    3. **미래 전망(MD&A)**: 경영진이 바라보는 향후 시장 전망 및 전략.
    4. **종합 평가**: 투자자 입장에서의 긍정적 요인과 부정적 요인 한 줄 요약.

    ---
    [보고서 텍스트 시작]
    {doc_text[:500000]} 
    [보고서 텍스트 끝]
    (참고: 텍스트가 너무 길 경우를 대비해 앞부분 50만 자만 전송합니다. Gemini 2.5 Pro를 쓴다면 전체 전송 가능)
    """

    try:
        response = model.generate_content(prompt)
        print("\n" + "="*30)
        print(f"   Gemini 분석 결과: {ticker}")
        print("="*30 + "\n")
        print(response.text)
        
    except Exception as e:
        print(f"[!] Gemini API Error: {e}")

## Gemini 분석

- 재무 하이라이트
- 주요 리스크 요인(Risk Factors)
- 미래 저망
- 종합 평가

In [30]:
# 예: Apple (AAPL) 분석
# NVDA, AAPL, GOOG, MSFT, AMZN, TSLA 등
#analyze_sec_filing("AAPL")
#analyze_sec_filing("NVDA")
analyze_sec_filing("GOOG")

--- GOOG 10-K 데이터 수집 중... ---
CIK : 1652044 0001652044
[*] 'GOOG'의 CIK: 0001652044
           accessionNumber  filingDate  reportDate        acceptanceDateTime  \
0     0001193125-25-291724  2025-11-21  2025-11-19  2025-11-21T20:04:57.000Z   
1     0001193125-25-291400  2025-11-21              2025-11-21T16:15:37.000Z   
2     0001193125-25-288144  2025-11-19  2025-11-17  2025-11-19T16:27:08.000Z   
3     0001950047-25-009101  2025-11-19              2025-11-19T16:22:49.000Z   
4     0001950047-25-009100  2025-11-19              2025-11-19T16:22:37.000Z   
...                    ...         ...         ...                       ...   
999   0001209191-22-024546  2022-04-15  2022-04-13  2022-04-15T21:35:08.000Z   
1000  0001209191-22-024472  2022-04-14  2022-04-12  2022-04-14T18:27:10.000Z   
1001  0001209191-22-024470  2022-04-14  2022-04-12  2022-04-14T18:23:50.000Z   
1002  0001209191-22-024443  2022-04-14  2022-04-12  2022-04-14T16:52:14.000Z   
1003  0001209191-22-024439  2022-04-1

## Claude Sonnet 4.5으로 생성한 소스

SEC EDGAR 문서 분석기 (완전판)
- 공식 SEC API 사용 (무료, API 키 불필요)
- 다양한 분석 방법 제공
- 10-K, 10-Q, 8-K 등 모든 Filing 지원

#### EDGAR Application Programming Interfaces (APIs)
공식 문서: https://www.sec.gov/search-filings/edgar-application-programming-interfaces

#### Accessing EDGAR Data
SEC 정책: https://www.sec.gov/os/accessing-edgar-data

#### Ticker-CIK 매핑 JSON
https://www.sec.gov/files/company_tickers.json

#### 회사의 모든 제출 문서
https://data.sec.gov/submissions/CIK{cik}.json

#### 회사의 재무 팩트 조회 (XBRL 데이터)
https://data.sec.gov/api/xbrl/companyfacts/CIK{cik}.json

#### 특정 용어의 데이터
taxonomy: XBRL taxonomy (us-gaap, ifrs-full, dei, srt)<br/>
concept: (AccountsPayableCurrent, Assets, Liabilities, StockholdersEquity, Revenues, NetIncomeLoss, EarningsPerShareBasic, CashAndCashEquivalentsAtCarryingValue, ...)<br/>
https://data.sec.gov/api/xbrl/companyconcept/CIK{cik}/{taxonomy}/{concept}.json

예: Apple Inc. US-GAAP, Revenues<br/>
https://data.sec.gov/api/xbrl/companyconcept/CIK0000320193/us-gaap/Revenues.json

#### Filing 문서
https://www.sec.gov/Archives/edgar/data/{cik}/{acc_no}/{primary_document}

In [31]:
import requests
import json
import pandas as pd
from typing import Dict, List, Optional
from datetime import datetime
import time
import re
from pathlib import Path

In [39]:
company_name = "Sayouzone"
email = "sjkim@sayouzone.com"

## SECEdgarClient



In [33]:
# =============================================================================
# SEC EDGAR API 클라이언트
# =============================================================================

class SECEdgarClient:
    """
    SEC EDGAR 공식 API 클라이언트
    
    API 키 불필요하지만 User-Agent는 필수
    SEC 정책: https://www.sec.gov/os/accessing-edgar-data
    """
    
    def __init__(self, company_name: str, email: str):
        """
        Args:
            company_name: 회사명
            email: 연락처 이메일
        """
        # SEC 요구사항: User-Agent 헤더 필수
        self.headers = {
            'User-Agent': f'{company_name} {email}',
            'Accept-Encoding': 'gzip, deflate'
        }
        
        self.base_url = "https://data.sec.gov"
        self.edgar_url = "https://www.sec.gov/cgi-bin/browse-edgar"
        
        # Rate limiting (초당 10개 요청 제한)
        self.rate_limit_delay = 0.1
    
    def _rate_limit(self):
        """Rate limiting 준수"""
        time.sleep(self.rate_limit_delay)
    
    def get_company_cik(self, ticker: str) -> Optional[str]:
        """
        티커 심볼로 CIK 번호 조회
        
        Args:
            ticker: 주식 티커 (예: 'AAPL')
        
        Returns:
            CIK 번호 (10자리)
        """
        # CIK 매핑 파일 다운로드
        # 회사이름, 티커, CIK
        url = "https://www.sec.gov/files/company_tickers.json"
        
        try:
            self._rate_limit()
            response = requests.get(url, headers=self.headers)
            response.raise_for_status()
            
            data = response.json()
            
            # 티커로 CIK 찾기
            for key, company in data.items():
                if company['ticker'].upper() == ticker.upper():
                    # CIK를 10자리로 패딩
                    cik = str(company['cik_str']).zfill(10)
                    print(f"✓ Found: {company['title']} (CIK: {cik})")
                    return cik
            
            print(f"✗ Ticker '{ticker}' not found")
            return None
        
        except Exception as e:
            print(f"Error getting CIK: {e}")
            return None
    
    def get_submissions(self, cik: str) -> Dict:
        """
        회사의 모든 제출 문서 조회
        
        Args:
            cik: CIK 번호
        
        Returns:
            제출 문서 정보 딕셔너리
        """
        # CIK를 10자리로 패딩
        cik = str(cik).zfill(10)
        
        url = f"{self.base_url}/submissions/CIK{cik}.json"
        
        try:
            self._rate_limit()
            response = requests.get(url, headers=self.headers)
            response.raise_for_status()
            
            return response.json()
        
        except Exception as e:
            print(f"Error getting submissions: {e}")
            return {}
    
    def get_company_facts(self, cik: str) -> Dict:
        """
        회사의 재무 팩트 조회 (XBRL 데이터)
        
        Args:
            cik: CIK 번호
        
        Returns:
            재무 팩트 딕셔너리
        """
        cik = str(cik).zfill(10)
        
        url = f"{self.base_url}/api/xbrl/companyfacts/CIK{cik}.json"
        
        try:
            self._rate_limit()
            response = requests.get(url, headers=self.headers)
            response.raise_for_status()
            
            return response.json()
        
        except Exception as e:
            print(f"Error getting company facts: {e}")
            return {}
    
    def get_company_concept(
        self,
        cik: str,
        taxonomy: str = "us-gaap",
        concept: str = "AccountsPayableCurrent"
    ) -> Dict:
        """
        특정 개념의 데이터 조회
        
        Args:
            cik: CIK 번호
            taxonomy: XBRL taxonomy (us-gaap, ifrs-full, dei, srt)
            concept: 개념 이름
        
        Returns:
            개념 데이터 딕셔너리
        """
        cik = str(cik).zfill(10)
        
        url = f"{self.base_url}/api/xbrl/companyconcept/CIK{cik}/{taxonomy}/{concept}.json"
        
        try:
            self._rate_limit()
            response = requests.get(url, headers=self.headers)
            response.raise_for_status()
            
            return response.json()
        
        except Exception as e:
            print(f"Error getting company concept: {e}")
            return {}
    
    def download_filing(
        self,
        accession_number: str,
        cik: str,
        primary_document: str
    ) -> Optional[str]:
        """
        Filing 문서 다운로드
        
        Args:
            accession_number: Accession number (하이픈 없이)
            cik: CIK 번호
            primary_document: 주 문서 파일명
        
        Returns:
            문서 HTML 내용
        """
        cik = str(cik).zfill(10)
        
        # Accession number 포맷팅 (하이픈 제거)
        acc_no = accession_number.replace("-", "")
        
        url = f"https://www.sec.gov/Archives/edgar/data/{cik}/{acc_no}/{primary_document}"
        
        try:
            self._rate_limit()
            response = requests.get(url, headers=self.headers)
            response.raise_for_status()
            
            return response.text
        
        except Exception as e:
            print(f"Error downloading filing: {e}")
            return None

## SECFilingAnalyzer



In [34]:
# =============================================================================
# SEC Filing 분석기
# =============================================================================

class SECFilingAnalyzer:
    """SEC Filing 문서 분석기"""
    
    def __init__(self, client: SECEdgarClient):
        self.client = client
    
    def get_recent_filings(
        self,
        cik: str,
        form_type: Optional[str] = None,
        limit: int = 10
    ) -> pd.DataFrame:
        """
        최근 Filing 목록 조회
        
        Args:
            cik: CIK 번호
            form_type: Filing 타입 (10-K, 10-Q, 8-K 등)
            limit: 반환할 최대 개수
        
        Returns:
            Filing 정보 DataFrame
        """
        submissions = self.client.get_submissions(cik)
        
        if not submissions:
            return pd.DataFrame()
        
        recent = submissions.get('filings', {}).get('recent', {})
        
        # DataFrame 생성
        df = pd.DataFrame({
            'accessionNumber': recent.get('accessionNumber', []),
            'filingDate': recent.get('filingDate', []),
            'reportDate': recent.get('reportDate', []),
            'form': recent.get('form', []),
            'primaryDocument': recent.get('primaryDocument', []),
            'primaryDocDescription': recent.get('primaryDocDescription', []),
        })
        
        # Form type 필터링
        if form_type:
            df = df[df['form'] == form_type]
        
        # 날짜순 정렬
        df = df.sort_values('filingDate', ascending=False)
        
        return df.head(limit)
    
    def extract_financial_metrics(self, cik: str) -> pd.DataFrame:
        """
        주요 재무 지표 추출
        
        Args:
            cik: CIK 번호
        
        Returns:
            재무 지표 DataFrame
        """
        facts = self.client.get_company_facts(cik)
        
        if not facts:
            return pd.DataFrame()
        
        metrics = []
        
        # US-GAAP 데이터 추출
        us_gaap = facts.get('facts', {}).get('us-gaap', {})
        
        # 주요 지표 목록
        important_metrics = [
            'Assets',
            'Liabilities',
            'StockholdersEquity',
            'Revenues',
            'NetIncomeLoss',
            'EarningsPerShareBasic',
            'CashAndCashEquivalentsAtCarryingValue'
        ]
        
        for metric in important_metrics:
            if metric in us_gaap:
                data = us_gaap[metric]
                label = data.get('label', metric)
                description = data.get('description', '')
                
                # USD 단위 데이터 추출
                units = data.get('units', {}).get('USD', [])
                
                if units:
                    # 가장 최근 값
                    latest = sorted(units, key=lambda x: x.get('end', ''), reverse=True)[0]
                    
                    metrics.append({
                        'Metric': label,
                        'Value': latest.get('val', 'N/A'),
                        'Date': latest.get('end', 'N/A'),
                        'Form': latest.get('form', 'N/A'),
                        'Description': description[:100]
                    })
        
        return pd.DataFrame(metrics)
    
    def analyze_revenue_trend(self, cik: str) -> pd.DataFrame:
        """
        매출 추이 분석
        
        Args:
            cik: CIK 번호
        
        Returns:
            매출 추이 DataFrame
        """
        revenue_data = self.client.get_company_concept(
            cik=cik,
            taxonomy="us-gaap",
            concept="Revenues"
        )
        
        if not revenue_data:
            return pd.DataFrame()
        
        # USD 단위 데이터 추출
        units = revenue_data.get('units', {}).get('USD', [])
        
        if not units:
            return pd.DataFrame()
        
        # DataFrame 생성
        df = pd.DataFrame(units)
        
        # 필요한 컬럼만 선택
        cols = ['end', 'val', 'fy', 'fp', 'form', 'filed']
        df = df[[col for col in cols if col in df.columns]]
        
        # 날짜순 정렬
        if 'end' in df.columns:
            df = df.sort_values('end', ascending=False)
        
        return df.head(20)
    
    def extract_text_from_filing(self, html_content: str) -> str:
        """
        HTML Filing에서 텍스트 추출
        
        Args:
            html_content: HTML 내용
        
        Returns:
            추출된 텍스트
        """
        # 간단한 HTML 태그 제거
        text = re.sub(r'<[^>]+>', ' ', html_content)
        
        # 여러 공백을 하나로
        text = re.sub(r'\s+', ' ', text)
        
        # XBRL 태그 제거
        text = re.sub(r'</?ix:[^>]+>', '', text)
        
        return text.strip()

## 파이프라인

회사에 대한 종합 데이터 분석 파이프라인

- ticker으로 CIK (Central Index Key) 확인
- 회사 제출 자료
- 최신 리포트
- 재무제표 추출
- 매출 추이

In [36]:
# =============================================================================
# 실전 클래스
# =============================================================================

class SECAnalysisPipeline:
    """완전한 SEC 분석 파이프라인"""
    
    def __init__(self, company_name: str, email: str):
        self.client = SECEdgarClient(company_name, email)
        self.analyzer = SECFilingAnalyzer(self.client)
    
    def analyze_company(self, ticker: str) -> Dict:
        """회사 종합 분석"""
        print(f"\n분석 중: {ticker}")
        print("-" * 50)
        
        # CIK 조회
        cik = self.client.get_company_cik(ticker)
        
        if not cik:
            return {"error": "Ticker not found"}
        
        # 기본 정보
        submissions = self.client.get_submissions(cik)
        
        # 최근 Filing
        recent_filings = self.analyzer.get_recent_filings(cik, limit=10)
        
        # 재무 지표
        metrics = self.analyzer.extract_financial_metrics(cik)
        
        # 매출 추이
        revenue = self.analyzer.analyze_revenue_trend(cik)
        
        return {
            "ticker": ticker,
            "cik": cik,
            "company_info": submissions,
            "recent_filings": recent_filings,
            "financial_metrics": metrics,
            "revenue_trend": revenue
        }

## 예제

- 기본 사용 예제
- 최근 Filing 조회 예제
- 재무 지표 분석 예제
- 매출 추이 분석 예제
- Filing 문서 다운로드 예제
- 회사 간 비교 예제
- 파일 저장 예제
- 완전한 파이프라인 예제

In [40]:
# =============================================================================
# 사용 예제
# =============================================================================

def example_basic_usage(ticker: str):
    """
    기본 사용 예제
    
    Args:
        ticker: 티커
    """
    print("\n" + "=" * 70)
    print("예제 1: 기본 사용법")
    print("=" * 70)
    
    # 클라이언트 생성
    client = SECEdgarClient(
        company_name=company_name,
        email=email
    )
    
    # 티커로 CIK 조회
    cik = client.get_company_cik(ticker)
    
    if cik:
        # 제출 문서 조회
        submissions = client.get_submissions(cik)
        
        print(f"\n회사명: {submissions.get('name')}")
        print(f"CIK: {submissions.get('cik')}")
        print(f"SIC: {submissions.get('sic')} - {submissions.get('sicDescription')}")
        print(f"티커: {', '.join(submissions.get('tickers', []))}")
        print(f"거래소: {', '.join(submissions.get('exchanges', []))}")
    
    print()


def example_recent_filings(ticker: str):
    """
    최근 Filing 조회 예제
    
    Args:
        ticker: 티커
    """
    print("\n" + "=" * 70)
    print("예제 2: 최근 Filing 조회")
    print("=" * 70)
    
    client = SECEdgarClient(company_name, email)
    analyzer = SECFilingAnalyzer(client)
    
    cik = client.get_company_cik(ticker)
    
    if cik:
        # 최근 10-K 문서
        filings = analyzer.get_recent_filings(cik, form_type="10-K", limit=5)
        
        print(f"\n최근 10-K Filings ({len(filings)} 개):")
        print(filings[['filingDate', 'form', 'primaryDocDescription']].to_string(index=False))
    
    print()


def example_financial_metrics(ticker: str):
    """
    재무 지표 분석 예제
    
    Args:
        ticker: 티커
    """
    print("\n" + "=" * 70)
    print("예제 3: 재무 지표 분석")
    print("=" * 70)
    
    client = SECEdgarClient(company_name, email)
    analyzer = SECFilingAnalyzer(client)
    
    cik = client.get_company_cik(ticker)
    
    if cik:
        # 재무 지표 추출
        metrics = analyzer.extract_financial_metrics(cik)
        
        if not metrics.empty:
            print(f"\n주요 재무 지표:")
            print(metrics.to_string(index=False))
    
    print()


def example_revenue_trend(ticker: str):
    """
    매출 추이 분석 예제
    
    Args:
         ticker: 티커
    """
    print("\n" + "=" * 70)
    print("예제 4: 매출 추이 분석")
    print("=" * 70)
    
    client = SECEdgarClient(company_name, email)
    analyzer = SECFilingAnalyzer(client)
    
    cik = client.get_company_cik(ticker)
    
    if cik:
        # 매출 추이
        revenue = analyzer.analyze_revenue_trend(cik)
        
        if not revenue.empty:
            print(f"\n매출 추이 (최근 10개):")
            print(revenue.head(10).to_string(index=False))
    
    print()


def example_download_filing(ticker: str):
    """
    Filing 문서 다운로드 예제
    
    Args:
        ticker: 티커
    """
    print("\n" + "=" * 70)
    print("예제 5: Filing 문서 다운로드")
    print("=" * 70)
    
    client = SECEdgarClient(company_name, email)
    analyzer = SECFilingAnalyzer(client)
    
    cik = client.get_company_cik(ticker)
    
    if cik:
        # 최근 10-K 가져오기
        filings = analyzer.get_recent_filings(cik, form_type="10-K", limit=1)
        
        if not filings.empty:
            filing = filings.iloc[0]
            
            print(f"\n다운로드 중: {filing['form']} ({filing['filingDate']})")
            
            # 문서 다운로드
            html_content = client.download_filing(
                accession_number=filing['accessionNumber'],
                cik=cik,
                primary_document=filing['primaryDocument']
            )
            
            if html_content:
                # 텍스트 추출
                text = analyzer.extract_text_from_filing(html_content)
                
                print(f"문서 크기: {len(html_content)} 문자")
                print(f"추출된 텍스트: {len(text)} 문자")
                print(f"\n내용 미리보기:\n{text[:500]}...")
    
    print()


def example_compare_companies(tickers: list[str]):
    """
    회사 간 비교 예제
    
    Args:
        tickers: 티커 리스트
    """
    print("\n" + "=" * 70)
    print("예제 6: 회사 간 비교")
    print("=" * 70)
    
    client = SECEdgarClient(company_name, email)
    analyzer = SECFilingAnalyzer(client)
    
    companies = tickers
    results = []
    
    for ticker in companies:
        cik = client.get_company_cik(ticker)
        
        if cik:
            submissions = client.get_submissions(cik)
            metrics = analyzer.extract_financial_metrics(cik)
            
            # 자산 정보 찾기
            assets_row = metrics[metrics['Metric'].str.contains('Assets', na=False)]
            
            results.append({
                'Ticker': ticker,
                'Company': submissions.get('name'),
                'Assets': assets_row.iloc[0]['Value'] if not assets_row.empty else 'N/A',
                'Assets Date': assets_row.iloc[0]['Date'] if not assets_row.empty else 'N/A'
            })
    
    df = pd.DataFrame(results)
    print(f"\n회사 비교:")
    print(df.to_string(index=False))
    
    print()


def example_save_to_file(ticker: str):
    """
    파일 저장 예제
    
    Args:
        ticker: 티커
    """
    print("\n" + "=" * 70)
    print("예제 7: 데이터 파일 저장")
    print("=" * 70)
    
    client = SECEdgarClient(company_name, email)
    analyzer = SECFilingAnalyzer(client)
    
    cik = client.get_company_cik(ticker)
    
    if cik:
        # 최근 Filing 조회
        filings = analyzer.get_recent_filings(cik, limit=20)
        
        # CSV로 저장
        output_file = "sec_filings_NVDA.csv"
        filings.to_csv(output_file, index=False)
        
        print(f"\n✓ Saved to {output_file}")
        print(f"  {len(filings)} filings saved")
        
        # 재무 지표 저장
        metrics = analyzer.extract_financial_metrics(cik)
        
        if not metrics.empty:
            metrics_file = "sec_metrics_NVDA.csv"
            metrics.to_csv(metrics_file, index=False)
            
            print(f"✓ Saved to {metrics_file}")
            print(f"  {len(metrics)} metrics saved")
    
    print()

전체 파이프라인 예제 실행

In [41]:
def example_full_pipeline(ticker: str):
    """
    완전한 파이프라인 예제
    
    Args:
        ticker: 티커
    """
    print("\n" + "=" * 70)
    print("예제 8: 완전한 분석 파이프라인")
    print("=" * 70)
    
    pipeline = SECAnalysisPipeline(company_name, email)
    
    # 회사 분석
    result = pipeline.analyze_company(ticker)
    
    if "error" not in result:
        info = result["company_info"]
        print(f"\n회사: {info.get('name')}")
        print(f"티커: {result['ticker']}")
        print(f"CIK: {result['cik']}")
        
        print(f"\n최근 Filing: {len(result['recent_filings'])} 개")
        print(result['recent_filings'][['filingDate', 'form']].head())
        
        print(f"\n재무 지표: {len(result['financial_metrics'])} 개")
        print(result['financial_metrics'].head())
    
    print()

모든 예제 실행

In [38]:
# =============================================================================
# 메인 실행
# =============================================================================

def main():
    """모든 예제 실행"""
    
    print("\n")
    print("🚀 SEC EDGAR 문서 분석기")
    print("=" * 70)
    print()
    
    print("⚠️  주의사항:")
    print("- SEC는 User-Agent 헤더를 요구합니다")
    print("- 초당 10개 요청 제한이 있습니다")
    print("- 회사명과 이메일을 실제 정보로 변경하세요")
    print()
    
    try:
        example_basic_usage("AAPL")
        example_recent_filings("TSLA")
        example_financial_metrics("MSFT")
        example_revenue_trend("GOOGL")
        example_download_filing("AAPL")
        example_compare_companies(["AAPL", "MSFT", "GOOGL"])
        example_save_to_file("NVDA")
        example_full_pipeline("AAPL")
        
        print("=" * 70)
        print("✅ 모든 예제 실행 완료!")
        print("=" * 70)
        
    except Exception as e:
        print(f"\n❌ 오류 발생: {e}")
        import traceback
        traceback.print_exc()

main()



🚀 SEC EDGAR 문서 분석기

⚠️  주의사항:
- SEC는 User-Agent 헤더를 요구합니다
- 초당 10개 요청 제한이 있습니다
- 회사명과 이메일을 실제 정보로 변경하세요


예제 1: 기본 사용법
✓ Found: Apple Inc. (CIK: 0000320193)

회사명: Apple Inc.
CIK: 0000320193
SIC: 3571 - Electronic Computers
티커: AAPL
거래소: Nasdaq


예제 2: 최근 Filing 조회
✓ Found: Tesla, Inc. (CIK: 0001318605)

최근 10-K Filings (5 개):
filingDate form primaryDocDescription
2025-01-30 10-K                  10-K
2024-01-29 10-K                  10-K
2023-01-31 10-K                  10-K
2022-02-07 10-K                  10-K
2021-02-08 10-K                  10-K


예제 3: 재무 지표 분석
✓ Found: MICROSOFT CORP (CIK: 0000789019)

주요 재무 지표:
                                      Metric        Value       Date Form                                                                                          Description
                                      Assets 636351000000 2025-09-30 10-Q Sum of the carrying amounts as of the balance sheet date of all assets that are recognized. Assets a
                      

==============================  
   Gemini 2.5 Pro 분석 결과: AAPL  
==============================

네, 전문 금융 애널리스트로서 Apple (AAPL)의 2025 회계연도 10-K 보고서를 분석하여 요청하신 내용을 요약해 드리겠습니다.

---

### **Apple (AAPL) 2025년 10-K 보고서 분석 요약**

#### **1. 재무 하이라이트**

2025 회계연도에 Apple은 견고한 성장세를 보였습니다.

*   **총 매출(Net Sales)**: 2025년 총 매출은 **4,161억 달러**로, 전년(3,910억 달러) 대비 약 6% 증가하며 성장세를 이어갔습니다.
*   **순이익(Net Income)**: 순이익은 **1,120억 달러**로, 전년(937억 달러) 대비 약 19.5% 크게 증가했습니다. 이는 2024년에 있었던 유럽연합(EU)의 일회성 세금 비용($102억) 기저효과가 반영된 결과입니다.
*   **제품별 성과**:
    *   **iPhone**: 매출이 4% 증가하며 여전히 핵심적인 수익원임을 입증했습니다.
    *   **서비스**: 광고, 앱스토어, 클라우드 서비스의 성장에 힘입어 매출이 14% 증가, 가장 높은 성장률을 기록하며 회사의 미래 성장 동력임을 보여주었습니다.
    *   **Mac**: 매출이 12% 증가하며 뚜렷한 회복세를 보였습니다.
    *   **Wearables, Home and Accessories**: 유일하게 매출이 4% 감소하며 다소 부진한 모습을 보였습니다.
*   **지역별 성과**: 미주, 유럽, 일본 등 대부분 지역에서 견조한 성장을 보였으나, **중화권(Greater China) 매출은 4% 감소**하여 지정학적 및 경쟁 심화의 영향이 나타나고 있음을 시사합니다.

#### **2. 주요 리스크 요인 (Risk Factors)**

경영진이 언급한 가장 중요하고 치명적인 리스크 3가지는 다음과 같습니다.

1.  **지정학적 리스크 및 무역 분쟁**: 보고서는 2025년부터 시행된 새로운 미국 관세(U.S. Tariffs)와 이에 따른 각국의 보복 조치를 명시적으로 언급합니다. 특히 **중국을 포함한 아시아 지역에 생산 및 공급망이 집중**되어 있어, 무역 분쟁이나 지정학적 긴장 고조가 부품 수급, 생산 비용, 제품 가격에 치명적인 영향을 미칠 수 있다고 경고합니다.
2.  **전 세계적인 규제 및 법적 소송 심화**: **EU의 디지털 시장법(DMA)**과 **미국 법무부(DOJ)의 반독점 소송**이 가장 큰 위협으로 부상했습니다. 이러한 규제는 Apple의 핵심 수익원인 앱스토어 비즈니스 모델을 근본적으로 바꿔야 할 수 있으며, 막대한 과징금 부과 및 사업 관행 변경을 강제할 수 있습니다. 이는 Apple의 강력한 생태계 경쟁력을 약화시킬 수 있는 리스크입니다.
3.  **치열한 시장 경쟁과 지속적인 혁신 압박**: 스마트폰, PC 등 주요 시장에서 Apple은 소수 시장 점유율을 차지하고 있으며, 경쟁사들은 공격적인 가격 정책과 제품 모방으로 끊임없이 압박하고 있습니다. Apple의 성공은 **지속적이고 시의적절한 혁신 제품 출시에 크게 의존**하므로, 만약 혁신에 실패하거나 시장의 기대를 충족시키지 못할 경우 경쟁 우위를 잃고 실적이 악화될 수 있습니다.

#### **3. 미래 전망 (MD&A)**

경영진은 미래 시장에 대해 신중하면서도 전략적인 방향성을 제시하고 있습니다.

*   **불확실한 거시 경제 및 관세 영향**: 경영진은 인플레이션, 금리, 환율 변동과 같은 거시 경제 상황과 함께, 특히 2025년부터 부과된 **미국 관세 및 무역 분쟁의 최종 영향은 여전히 불확실하다**고 언급하며, 이것이 향후 실적에 중대한 변수가 될 수 있음을 인정하고 있습니다.
*   **서비스 부문의 지속적인 성장 전략**: 재무 성과에서 확인되듯이, Apple은 **서비스 부문을 핵심 성장 동력**으로 보고 있습니다. 광고, 앱스토어, 클라우드 및 구독 서비스의 성장을 통해 하드웨어 판매에 대한 의존도를 줄이고, 고수익성의 안정적인 수익원을 확대하려는 전략을 지속할 것으로 보입니다.
*   **공격적인 연구개발(R&D) 투자**: 2025년 R&D 비용은 전년 대비 10% 증가한 **345억 달러**에 달합니다. 이는 AI와 같은 신기술, 새로운 제품 카테고리 개발에 막대한 투자를 하고 있음을 보여주며, 치열한 기술 경쟁에서 우위를 점하고 미래 성장 기회를 창출하려는 강력한 의지를 나타냅니다.

#### **4. 종합 평가 (투자자 관점)**

*   **긍정적 요인**: 견고한 서비스 부문 성장과 강력한 생태계를 통한 안정적인 수익 창출 능력.
*   **부정적 요인**: 중화권 매출 감소, 심화되는 지정학적 리스크 및 전 세계적인 반독점 규제 강화.

==============================  
   Gemini 2.5 Pro 분석 결과: NVDA  
==============================  

네, 전문 금융 애널리스트로서 NVIDIA(NVDA)의 2025년 회계연도 10-K 보고서를 분석하여 핵심 내용을 아래와 같이 요약해 드립니다.

---

### **NVIDIA(NVDA) 10-K 보고서 분석 및 요약 (2025년 회계연도 기준)**

#### **1. 재무 하이라이트**

NVIDIA는 생성형 AI 시장의 폭발적인 성장에 힘입어 경이적인 재무 성과를 기록했습니다.

*   **매출**: 2025년 회계연도(FY2025) 총매출은 **1,305억 달러**로, 전년(FY2024)의 609억 달러 대비 **114% 증가**하며 폭발적인 성장세를 보였습니다.
*   **순이익**: FY2025 순이익은 **729억 달러**로, 전년의 298억 달러 대비 **145% 급증**했습니다. 이는 매출 성장률을 상회하는 수치로, 높은 수익성을 증명합니다.
*   **성장 동력**: 이러한 성과는 **데이터센터 부문**이 주도했습니다. 데이터센터 매출은 전년 대비 **142% 증가**했으며, 이는 대규모 언어 모델(LLM) 및 생성형 AI 애플리케이션을 위한 'Hopper' 아키텍처 GPU에 대한 압도적인 수요 때문입니다.

**결론**: AI 시장의 개화에 따른 데이터센터 수요가 전사적 성장을 견인하며, 매출과 이익 모두 전례 없는 수준으로 급증하는 추세입니다.

#### **2. 주요 리스크 요인 (Risk Factors)**

경영진은 다음과 같은 3가지 리스크를 사업의 지속 가능성을 위협하는 가장 치명적인 요인으로 꼽고 있습니다.

1.  **수출 통제 및 지정학적 리스크 (Export Controls & Geopolitical Risks)**:
    미국 정부의 중국 및 특정 중동 국가에 대한 고성능 AI 칩(A100, H100, Blackwell 시리즈 등) 수출 통제 조치가 가장 심각한 리스크입니다. 이로 인해 잠재적으로 가장 큰 시장 중 하나인 중국에서의 사업이 크게 위축되었으며, 규제가 강화될 경우 매출과 경쟁력에 장기적인 타격을 줄 수 있습니다. 또한, 대만 등 핵심 공급망이 위치한 지역의 지정학적 긴장도 큰 위협 요인입니다.

2.  **공급망 의존성 및 수요 예측 실패 리스크 (Supply Chain Dependency & Demand Forecasting Failure)**:
    NVIDIA는 팹리스(fabless) 기업으로서 TSMC, 삼성 등 소수의 파운드리 업체에 웨이퍼 생산을 전적으로 의존합니다. 특정 공급업체에 대한 과도한 의존은 생산 차질 발생 시 즉각적인 타격으로 이어질 수 있습니다. 또한, AI 시장의 수요 변동성이 매우 커서, 수요를 과소 예측하면 막대한 매출 기회를 상실하고, 과대 예측하면 막대한 재고 평가손실 및 구매 약정 위약금을 물게 될 위험이 공존합니다.

3.  **심화되는 경쟁 (Intensifying Competition)**:
    AMD, Intel과 같은 전통적인 경쟁사뿐만 아니라, 주요 고객인 대형 클라우드 서비스 제공업체(CSP)들(Google, Amazon, Microsoft 등)이 자체 AI 칩 개발에 나서고 있는 것이 새로운 위협입니다. 이들이 자체 칩 사용 비중을 늘릴 경우 NVIDIA의 가장 큰 매출처인 데이터센터 시장의 성장세가 둔화될 수 있습니다.

#### **3. 미래 전망 (MD&A - 경영진의 진단 및 분석)**

경영진은 향후 시장을 매우 긍정적으로 전망하며, 기술 리더십을 유지하기 위한 명확한 전략을 제시하고 있습니다.

*   **시장 전망**: 생성형 AI와 대규모 언어 모델(LLM)의 발전이 이제 시작 단계에 있으며, 이는 전 산업에 걸쳐 컴퓨팅 수요를 폭발적으로 증가시킬 것으로 보고 있습니다. 경영진은 '데이터센터가 새로운 컴퓨팅의 단위'가 되었다고 진단하며, 개별 칩이 아닌 데이터센터 규모의 통합 플랫폼에 대한 수요가 지속될 것으로 전망합니다.
*   **핵심 전략**:
    1.  **AI 플랫폼 리더십 강화**: 단순한 하드웨어 판매를 넘어, CUDA 소프트웨어, 네트워킹(InfiniBand, Ethernet) 솔루션을 포함한 **'풀스택(Full-stack) AI 컴퓨팅 플랫폼'**을 제공하여 강력한 생태계를 구축하고 고객 이탈을 방지하는 전략을 고수합니다.
    2.  **지속적인 기술 혁신**: 시장 지배력을 유지하기 위해 매년 새로운 아키텍처를 출시하는 빠른 혁신 주기를 가져가고 있습니다. 성공적인 'Hopper' 아키텍처에 이어 차세대 **'Blackwell' 플랫폼**의 양산을 시작했으며, 이를 통해 경쟁사와의 기술 격차를 계속 벌려 나갈 계획입니다.
    3.  **규제 대응**: 수출 통제에 대응하기 위해 규제를 준수하는 중국 시장용 제품을 개발하는 등 유연한 제품 전략을 구사하고 있습니다.

#### **4. 종합 평가 (투자자 관점)**

*   **긍정적 요인**: 생성형 AI 시장의 폭발적인 성장에 따른 독점적인 시장 지위와 CUDA 생태계를 기반으로 한 강력한 기술 해자(moat).
*   **부정적 요인**: 미-중 기술 패권 경쟁에 따른 직접적인 수출 통제 리스크와 소수 아시아 파운드리에 집중된 공급망 의존도.

==============================  
   Gemini 2.5 Pro 분석 결과: GOOG  
==============================

## Alphabet (GOOG) SEC 10-K 보고서 분석 (2024년 회계연도 기준)

전문 금융 애널리스트로서 Alphabet Inc. (GOOG)의 2024년 회계연도(2024년 12월 31일 마감) 10-K 보고서를 분석한 결과는 다음과 같습니다.

### 1. 재무 하이라이트

Alphabet은 견고하고 지속적인 성장세를 보였습니다. 특히 순이익의 증가율이 매출 증가율을 크게 상회하며 수익성이 개선된 점이 주목됩니다.

*   **총 매출**: 2024년 3,500억 달러로, 전년(3,074억 달러) 대비 **14% 증가**했습니다. 이는 구글 서비스(특히 검색)의 꾸준한 성장과 구글 클라우드의 가파른 성장에 기인합니다.
*   **순이익**: 2024년 1,001억 달러로, 전년(738억 달러) 대비 **36% 급증**했습니다. 이는 매출 성장과 더불어 비용 관리 및 운영 효율화 노력이 반영된 결과로 보입니다.
*   **부문별 성과**:
    *   **구글 서비스 (광고, Android, Chrome 등)**: 매출이 3,049억 달러로 전년 대비 12% 증가하며 전체 성장을 견인했습니다.
    *   **구글 클라우드**: 매출이 432억 달러로 전년 대비 **31%의 높은 성장률**을 기록했으며, 영업이익 또한 17억 달러에서 61억 달러로 크게 증가하여 수익성 개선에 기여했습니다.
    *   **Other Bets (Waymo 등 신사업)**: 여전히 44억 달러의 영업손실을 기록하며 장기 투자 단계에 머물러 있습니다.

**추세 요약**: 광고 사업의 견고한 기반 위에서 클라우드 사업이 새로운 성장 동력으로 확실히 자리매김했으며, 전반적인 수익성 개선이 뚜렷하게 나타나고 있습니다.

### 2. 주요 리스크 요인 (Risk Factors)

경영진이 언급한 가장 치명적인 리스크 3가지는 다음과 같습니다.

1.  **광고 매출에 대한 높은 의존도 (Dependency on Advertising)**:
    *   2024년에도 전체 매출의 75% 이상을 광고에서 창출하고 있어, 경기 침체로 인한 광고주 지출 감소, 온라인 광고 차단 기술의 확산, 그리고 제3자 쿠키 지원 중단과 같은 개인정보보호 정책 변화에 매우 취약합니다. 이는 Alphabet의 핵심 수익 모델을 직접적으로 위협하는 가장 근본적인 리스크입니다.

2.  **치열한 경쟁, 특히 AI 분야 (Intense Competition, especially in AI)**:
    *   검색, 광고, 클라우드, AI 등 모든 사업 분야에서 기존의 거대 기업 및 신생 스타트업과 치열하게 경쟁하고 있습니다. 특히 생성형 AI 기술은 산업의 판도를 바꾸는 '거대한 플랫폼 변화'로, 경쟁사들이 더 우수하거나 비용 효율적인 AI 제품을 먼저 출시할 경우 시장 지위를 잃을 수 있습니다. AI 기술 개발 및 인프라 구축에 대한 막대한 투자가 필수적이며, 이 경쟁에서 뒤처질 경우 비즈니스에 심각한 타격을 입을 수 있습니다.

3.  **전 세계적인 규제 강화 및 법적 소송 (Regulatory Scrutiny and Legal Liability)**:
    *   미국 법무부(DOJ), 유럽연합 집행위원회(EC) 등이 제기한 여러 반독점 소송에 직면해 있습니다. 검색, 광고 기술, 안드로이드 운영체제 및 플레이 스토어 등 핵심 비즈니스 모델이 모두 소송 대상입니다. 패소 시 막대한 벌금은 물론, 사업 모델의 근본적인 변경(구조적 분할 포함)을 강요받을 수 있습니다. 또한, 데이터 개인정보보호, AI, 콘텐츠 관리 등에 대한 새로운 법률이 계속 도입되고 있어 규제 준수 비용과 법적 불확실성이 지속적으로 증가하고 있습니다.

### 3. 미래 전망 (MD&A - 경영진의 분석 및 토론)

경영진은 인공지능(AI)을 미래 성장의 핵심으로 보고, 전사적 역량을 집중하는 전략을 명확히 하고 있습니다.

*   **AI 중심의 사업 재편 (AI-First Company)**: 경영진은 AI를 '심오한 플랫폼 변화'로 규정하고, 자체 개발한 **Gemini 모델**을 검색, 유튜브, 클라우드, 안드로이드 등 모든 핵심 제품에 통합하여 사용자 경험을 혁신하고 있습니다. 특히 AI 기술을 통해 더 복잡한 질문에 답하고 생산성을 높이는 도구를 제공하는 것을 최우선 과제로 삼고 있습니다.
*   **클라우드 사업의 AI 성장 동력화**: 기업 고객들이 AI 혁신을 주도할 수 있도록 Vertex AI 플랫폼과 Gemini for Google Cloud를 통해 최첨단 AI 인프라와 모델을 제공하는 데 주력하고 있습니다. 이는 클라우드 사업의 성장을 더욱 가속화할 핵심 전략입니다.
*   **지속적인 인프라 투자**: AI 제품 및 서비스 지원을 위해 서버, 네트워크 장비, 데이터센터 등 기술 인프라에 대한 투자를 2024년보다 더욱 확대할 계획입니다. 이는 AI 경쟁력을 유지하기 위한 필수적인 투자로 보고 있습니다.
*   **광고 외 수익원 다각화**: 클라우드, 소비자 구독 서비스(유튜브 프리미엄 등), 하드웨어 기기(Pixel 등) 매출의 지속적인 성장을 기대하고 있습니다. 다만, 이들 사업은 전통적인 광고 사업보다 마진이 낮을 수 있다는 점을 인지하고 있습니다.

**전략 요약**: 경영진은 AI 기술 리더십을 확보하고 이를 모든 서비스에 성공적으로 통합하는 것이 Alphabet의 미래를 결정할 것이라고 보고 있으며, 이를 위해 대규모의 장기적인 투자를 지속할 것임을 분명히 했습니다.

### 4. 종합 평가

**긍정적 요인**: AI를 필두로 한 핵심 광고 및 클라우드 사업의 견조한 성장세와 뛰어난 수익성 개선.
**부정적 요인**: 광고에 편중된 수익 구조와 전 세계적으로 심화되는 반독점 규제 및 법적 리스크.

==============================  
   Gemini 2.5 Pro 분석 결과: GOOG  
==============================  

네, 전문 금융 애널리스트로서 Alphabet Inc.(GOOG)의 2025년 3분기 실적 보고서(Form 10-Q)를 분석하여 핵심 내용을 한국어로 요약해 드리겠습니다.

---

### **Alphabet Inc. (GOOG) 2025년 3분기 실적 분석 보고서**

본 문서는 Alphabet이 2025년 9월 30일 마감된 분기에 대해 SEC에 제출한 Form 10-Q 보고서를 기반으로 작성되었습니다.

#### **1. 재무 하이라이트**

2025년 3분기 실적은 시장 예상을 상회하는 견조한 성장세를 보였으며, 특히 구글 클라우드 부문의 성장이 두드러졌습니다.

*   **총 매출**: 2025년 3분기 매출은 **1,023억 달러**로, 전년 동기(883억 달러) 대비 **16% 증가**했습니다. 이는 구글 서비스(광고, 구독 등)와 구글 클라우드 양 부문의 고른 성장에 기인합니다.
*   **순이익**: 3분기 순이익은 **350억 달러**로, 전년 동기(263억 달러) 대비 **33% 급증**했습니다. 이는 매출 성장과 더불어 비상장 지분 증권(non-marketable equity securities) 평가 이익 등 영업 외 수익(OI&E)이 크게 증가한 영향입니다.
*   **사업 부문별 성과**:
    *   **구글 서비스 (Google Services)**: 매출 871억 달러 (전년 동기 대비 14% 증가). 검색 광고와 유튜브 광고가 성장을 견인했습니다.
    *   **구글 클라우드 (Google Cloud)**: 매출 152억 달러 (전년 동기 대비 **34% 증가**). 인프라 서비스 수요에 힘입어 폭발적인 성장세를 이어가며 수익성 개선에 크게 기여했습니다.
    *   **아더 베츠 (Other Bets)**: 매출 3.4억 달러로 소폭 감소했으며, 영업 손실은 14억 달러로 확대되었습니다.

#### **2. 주요 리스크 요인 (Risk Factors)**

보고서는 미국, 유럽 등 전 세계적으로 심화되는 규제 압박을 가장 큰 리스크로 명시하고 있습니다. 경영진이 가장 심각하게 우려하는 3가지 리스크는 다음과 같습니다.

1.  **광고 기술(Ad-Tech) 관련 반독점 소송**: 미국 법무부(DOJ), 주 법무장관, 유럽연합 집행위원회(EC) 등 다수의 규제 기관이 구글의 광고 기술 사업 관행을 문제 삼아 소송을 제기했습니다. 특히 EC는 2025년 9월, **30억 유로(약 35억 달러)의 벌금을 부과**했으며, DOJ가 제안한 구제책에는 **"구조적 분리" 가능성**이 포함되어 있어 사업의 근간을 위협할 수 있는 중대한 리스크입니다.
2.  **검색 및 앱스토어(Google Play) 독점 관련 규제**: 미국과 유럽에서 구글의 검색 서비스 배포 방식과 안드로이드 및 구글 플레이 스토어 운영에 대한 반독점 소송이 진행 중입니다. 미국 법원은 이미 구글의 검색 배포 관행에 대해 일부 위법 판결을 내렸으며, 이는 향후 구글의 기본 검색 엔진 지위와 비즈니스 모델에 직접적인 타격을 줄 수 있습니다.
3.  **유럽 디지털 시장법(DMA) 준수 리스크**: 2024년 3월부터 시행된 유럽 DMA 규정 준수와 관련하여 EC는 구글 플레이 및 검색 서비스에 대해 조사를 개시했으며, **"예비적 위반" 결론**을 내렸습니다. DMA는 거대 플랫폼 기업의 시장 지배력 남용을 막기 위한 강력한 법안으로, 위반 시 막대한 벌금과 사업 방식 변경 명령이 뒤따를 수 있습니다.

#### **3. 미래 전망 (MD&A - 경영진의 설명 및 분석)**

경영진은 인공지능(AI)을 중심으로 한 기술 리더십 확보와 클라우드 사업 확장에 대한 강력한 의지를 표명했습니다.

*   **AI 인프라에 대한 대규모 투자**: 보고서는 "2026년에는 AI 제품 및 서비스 지원을 위해 기술 인프라(서버, 데이터센터 등)에 대한 투자를 2025년 대비 **상당히(significantly) 늘릴 것**"이라고 명시했습니다. 이는 AI 시대의 컴퓨팅 수요에 대응하기 위한 선제적이고 공격적인 투자 전략을 시사합니다.
*   **클라우드 사업 강화 전략**: 320억 달러 규모의 클라우드 보안 플랫폼 **'위즈(Wiz)' 인수**를 추진하는 등, 구글 클라우드 사업의 경쟁력 강화를 위한 전략적 M&A를 지속하고 있습니다. 이는 단순한 인프라 제공을 넘어 고부가가치 서비스, 특히 보안과 AI 솔루션 분야에서 시장 지배력을 확대하려는 의도로 분석됩니다.
*   **견고한 핵심 사업 유지**: 검색, 유튜브 등 핵심 광고 사업의 안정적인 성장세를 바탕으로 확보된 현금 흐름을 AI, 클라우드 등 미래 성장 동력에 재투자하는 선순환 구조를 유지할 계획입니다.

#### **4. 종합 평가 (투자자 관점)**

*   **긍정적 요인**: 견조한 광고 사업과 폭발적으로 성장하는 클라우드 부문이 실적을 견인하고 있으며, AI 분야에 대한 대규모 선제적 투자는 미래 성장 동력을 확보하고 있습니다.
*   **부정적 요인**: 미국과 유럽에서 동시다발적으로 진행되는 반독점 소송은 핵심 사업 모델을 근본적으로 뒤흔들 수 있는 구조적 변화 강제 및 막대한 벌금 부과라는 심각한 불확실성을 안고 있습니다.